In [1]:
%matplotlib notebook
import numpy as np
from scipy.optimize import curve_fit

import ipywidgets as widgets
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy import units as u

# Telescope Position Data

In [2]:
lst, alt, az, ra, dec = np.loadtxt('data/telescope_orientation.csv', delimiter=',', unpack=True)
ha = lst - ra

In [3]:
fig = plt.figure(figsize=(7, 7))
frame = fig.add_subplot(1, 1, 1, projection ='3d')

frame.grid(True, alpha=0.5)
# frame.set_xlim(-5, 5)
# frame.set_ylim(-5, 5)
# frame.set_zlim(0, 5)

frame.set_xlabel('HA (h)')
frame.set_ylabel('Dec (deg)')
frame.set_zlabel('Az (deg)')

plt.plot(ha, dec, az, color='black', ls='', marker='o', ms=2)
plt.show()

<IPython.core.display.Javascript object>

# Computing HA

In [4]:
def compute_ha(ra, date):
    observing_location = EarthLocation(lat=53.240243*u.deg, lon=6.53651*u.deg) # Blaauw Observatory
    observing_time = Time(date, scale='utc', location=observing_location)
    lst = observing_time.sidereal_time('mean')
    
    print('LST = {:.5f} h'.format(lst.hour))
    
    return lst.hour - ra

## 13:17

In [5]:
# ra_ex = 6 + 2 * 1/60 + 7 * 1/3600
# dec_ex = 50 + 50 * 1/60
# ha_ex = compute_ha(ra_ex, datetime(2021, 5, 14, hour=11, minute=17))

# print('HA = {:.2f} h = {:.2f} deg & Dec = {:.2f} deg'.format(ha_ex, ha_ex * 15, dec_ex))

## 13:19

In [6]:
# ra_ex = 6 + 28 * 1/60 + 1 * 1/3600
# dec_ex = 7 + 50 * 1/60 # deg
# ha_ex = compute_ha(ra_ex, datetime(2021, 5, 14, hour=11, minute=19)) 

# print('HA = {:.2f} h = {:.2f} deg & Dec = {:.2f} deg'.format(ha_ex, ha_ex * 15, dec_ex))

# The Altitude

In [7]:
def altitude(ha, dec):
    """Returns altitude (in Grunn) in deg."""
    lat = np.radians(53.24)
    dec = np.radians(dec)
    ha  = np.radians(ha)
    
    term_1 = np.sin(dec)*np.sin(lat)
    term_2 = np.cos(dec)*np.cos(lat)*np.cos(ha)
    
    a = np.degrees(np.arcsin(term_1 + term_2))
    
    return a

# Visualizing Obstruction Data

In [8]:
from joblib import Parallel, delayed, dump, load

file_name = lambda n: '../scripts/data/blocked_grid_az_' + str(n) + '.joblib'
azs = np.arange(0, 360, 1)

In [9]:
cube = []

for n_az in azs:
    with open(file_name(n_az), 'rb') as f:
        data = load(f)
        cube.append(data)
#         print('az = {:.2f} deg w/ shape = {}'.format(n_az, data.shape))

cube = np.array(cube)

In [10]:
# _az = np.linspace(0, 359, 180)
# _h = np.linspace(-180, 180, 181)
# _dec = np.linspace(-90, 90, 91)

_az = np.linspace(0, 359, 360)
_h = np.linspace(-179, 180, 360) 
_dec = np.linspace(-90, 90, 181)

az, h, dec = np.meshgrid(_az, _h, _dec, indexing='ij')

alt = altitude(h, dec)

cond_1 = cube == cube.min()
cond_2 = alt >= 15

cond = cond_1 & cond_2

h_selected = h[cond]
h_out = np.where(h_selected < 0, h_selected + 360, h_selected) # offset hour angle

dec_selected = dec[cond]
az_selected = az[cond]

In [11]:
# h_lin = h.ravel()
# dec_lin = dec.ravel()
# az_lin = az.ravel()

# b = cube.ravel()

# coords = np.column_stack((h_lin, dec_lin))

In [12]:
h_range = np.arange(h_selected.min(), h_selected.max() + 1, 1)
h_range_shifted = np.arange(h_out.min(), h_out.max() + 1, 1)
dec_range = np.arange(dec_selected.min(), dec_selected.max() + 1, 1)

The following cell is meant for extracting an _azimuth surface_ from the data set.

In [13]:
# coords = []
# azimuths = []

# for h in h_range:
#     for dec in dec_range:
#         cond = np.isclose(h_out, h)&np.isclose(dec_selected, dec)
        
#         az = None
        
#         try:
#             az = az_selected[cond].max()
#         except:
#             pass
        
#         if az is not None:
#             coords.append((h, dec))
#             azimuths.append(az)

# coords = np.array(coords)
# azimuths = np.array(azimuths)

## Plot

In [14]:
fig = plt.figure(figsize=(7, 7))
frame = fig.add_subplot(1, 1, 1, projection ='3d')

frame.grid(True, alpha=0.5)

frame.set_xlabel('HA (deg)')
frame.set_ylabel('Dec (deg)')
frame.set_zlabel('Dome Az (deg)')

frame.plot(h_selected[::20], dec_selected[::20], az_selected[::20], color='black', ls='', alpha=0.5, marker='o', ms=2, label='0% obstruction')

# frame.plot(h_sel, dec_sel, 180, color='xkcd:salmon', ls='', alpha=0.5, marker='o', ms=2)
# plt.plot(h_selected[np.isclose(az_selected, 359.)], dec_selected[np.isclose(az_selected, 359.)], az_selected[np.isclose(az_selected, 359.)], color='purple', ls='', alpha=0.5, marker='o', ms=2, label='$A_z=359$ deg')

# hs, ds = coords.T
# frame.plot(hs, ds, azimuths, color='xkcd:black', ls='', alpha=0.8, marker='o', ms=2, label='$\max(A_z)$ w/ 0% obstruction')

# frame.view_init(elev=90, azim=90)

frame.legend(fontsize=13)

plt.show()

<IPython.core.display.Javascript object>

## Optimal $A_d$

Assume an HA and Dec of resp. $0^h0^m0^s$ and $0\ \text{deg}$

In [15]:
def delta_h(h, h_0):    
    return h - h_0    

In [16]:
def delta_h(h, h_0, m=360):
    tmp = (h % m) - (h_0 % m)
    return tmp % m

In [17]:
def optimal_az(az_options, h, dec):
    dec_sel = np.argwhere(np.isclose(dec_selected, dec)).ravel()
    
    azimuths = []
    delta_hs = []
    
    for az in az_options:
        az_sel = np.argwhere(np.isclose(az_selected, az)).ravel()
        
        indices = np.intersect1d(az_sel, dec_sel)
    
        # Extract the range of possible HAs
        hs = h_selected[indices]
        
        if hs.size:
            dh = delta_h(hs, h)
            
            delta_hs.append(dh.max())
            azimuths.append(az)
    
    azimuths = np.array(azimuths)
    delta_hs = np.array(delta_hs)
    
    return azimuths[delta_hs.argmax()], delta_hs.max()

## Selecting $A_d$ Values Given $(h,\delta)$ Pairs

In [18]:
# @widgets.interact(ha=(-179, 180, 1.0), dec=(-90, 90, 1.0))
# def update(ha=0.0, dec=0.0):
#     # Select indices according to the given ha/dec
#     dec_sel = np.argwhere(np.isclose(dec_selected, dec)).ravel()
#     h_sel   = np.argwhere(np.isclose(h_selected, ha)).ravel()
    
#     indices = np.intersect1d(h_sel, dec_sel)
    
#     # Find the range of azimuth values
#     az = az_selected[indices]
    
#     # Display the results
#     if not az.size:
#         print('No azimuths available!')
#     else:
#         print('Available az (deg): {:} az points\n'.format(az.size))
    
#         for a in az:
#             print('{:.0f}'.format(a), end=', ')
        
#         hd_cur = [ha, dec]
#         az_opt, dh = optimal_az(az, *hd_cur)
        
#         print('\n\nThe optimal value is {:.0f} deg'.format(az_opt))

In [23]:
coords = []
optimal_azimuths = []
hour_angle_diff = []

for h in h_range_shifted:
    for dec in dec_range:
        # Select indices according to the given ha/dec
        dec_sel = np.argwhere(np.isclose(dec_selected, dec)).ravel()
        h_sel   = np.argwhere(np.isclose(h_out, h)).ravel()
        
#         h_sel   = np.argwhere(np.isclose(h_selected, h)).ravel()

        indices = np.intersect1d(h_sel, dec_sel)

        # Find the range of azimuth values
        az = az_selected[indices]

        # Display the results
        if az.size > 0:
            hd_cur = [h, dec]
            az_opt, dh = optimal_az(az, *hd_cur)
            
            print(dh)
            
            coords.append(hd_cur)
            optimal_azimuths.append(az_opt)
            hour_angle_diff.append(dh)

coords = np.array(coords)
optimal_azimuths = np.array(optimal_azimuths)
hour_angle_diff = np.array(hour_angle_diff)

359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0
359.0


KeyboardInterrupt: 

In [ ]:
fig = plt.figure(figsize=(7, 7))
frame = fig.add_subplot(1, 1, 1, projection ='3d')

frame.grid(True, alpha=0.5)

frame.set_xlabel('HA (deg)')
frame.set_ylabel('Dec (deg)')
frame.set_zlabel('Dome Az (deg)')

frame.plot(coords[:, 0], coords[:, 1], optimal_azimuths, color='black', ls='', alpha=0.5, marker='o', ms=2)

plt.show()

## Store the Data

Zero obstruction:

In [ ]:
sel = np.column_stack([h_selected, dec_selected, az_selected])
np.savetxt('zero_obstruction_1_deg_seperation.csv', sel, delimiter=',')

Optimal azimuths:

In [ ]:
opt = np.column_stack([coords[:, 0], coords[:, 1], optimal_azimuths, hour_angle_diff/15])
np.savetxt('optimal_az_v2.csv', opt, delimiter=',')

# Above 15 deg

In [ ]:
h, dec = np.meshgrid(_h, _dec, indexing='ij')

In [ ]:
alt = altitude(h, dec)
cond = alt >= 15

In [ ]:
# %matplotlib inline

fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111)

h_sel = h[cond]
dec_sel = dec[cond]

# plt.scatter(h_sel, dec_sel, s=2)


plt.show()

In [ ]:
decs = []
for h in _h:
    min_dec = dec_sel[h_sel == h].min()
    
    decs.append(min_dec)

decs = np.array(decs)

In [ ]:
plt.plot(_h, decs)

In [ ]:
def cos_func(x, a, b, c):
    return a*np.cos(b*x)+c

p_init = (-40, 0.05, 20)
ps, cov = curve_fit(cos_func, _h, decs, p_init)

In [ ]:
h_range = np.linspace(-180, 180, 500)

In [ ]:
plt.plot(h_selected[::4], dec_selected[::4], color='green', ls='', marker='o', ms=1)

# plt.scatter(_h, decs, s=2, alpha=0.5, color='blue')
plt.plot(h_sel, dec_sel, color='blue', ls='', marker='o', ms=1)
plt.plot(h_range, cos_func(h_range, *ps), color='black')